El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

# Etapas del proyecto 

1. Importaremos las librerias y prepararemos los datos.
   * 1.1 Importaremos las librerias.
   * 1.2 Leeremos y revisaremos los datos.
2. Limpiaremos  y prepararemos los datos para entrenar los modelos.
   * 2.1 Limpieza de Datos
   * 2.2 Preparación de Datos.
3. Entrenar modelos.
4. Analisis de los 3 mejores modelos .
5. Conclusion.

# 1.1 Importación de Librerias

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import time

# 1.2 Leeremos y Revisaremos los datos

In [2]:
df = pd.read_csv('/datasets/car_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
# Todas las columnas minusculas
df.columns = df.columns.str.lower()
columnas_inecesarias = ['datecrawled', 'registrationmonth', 'datecreated', 'numberofpictures', 'postalcode', 'lastseen']
df = df.drop(columnas_inecesarias, axis=1)
df.head()

,price,vehicletype,registrationyear,gearbox,power,model,mileage,fueltype,brand,notrepaired
0,480,NaN,1993,manual,0,golf,150000,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no


# 2.1 Limpieza de los Datos

**Limpieza de los valores incongruentes en la columna de `registrationyear`.**

In [5]:
# Identificar filas con valores atípicos en 'registrationyear'
valores_atipicos = df[(df['registrationyear'] < 1910) | (df['registrationyear'] > 2020)]

# Filtrar filas con valores válidos en el rango de años 1910-2020
valores_validos = df[(df['registrationyear'] >= 1910) & (df['registrationyear'] <= 2020)]

# Iterar sobre las filas con valores atípicos
for idx, fila_atipica in valores_atipicos.iterrows():
    # Filtrar filas con características similares a la fila atípica
    filas_similares = valores_validos[
        (valores_validos['brand'] == fila_atipica['brand']) &
        (valores_validos['model'] == fila_atipica['model'])       
    ]
    # Calcular la mediana de 'registrationyear' para filas similares
    mediana_similares = filas_similares['registrationyear'].median()
    # Imputar la mediana como el nuevo valor para la fila atípica
    df.loc[idx, 'registrationyear'] = mediana_similares

# Verificar que el cambio se haya aplicado correctamente
print(df['registrationyear'].min())

1910.0


**Limpieza de los datos faltantes en las columnas `vehicletype, gearbox, model, fueltype y notrepaired`.**

In [6]:
# Lista de columnas con datos faltantes
columnas_faltantes = ['vehicletype', 'gearbox', 'model', 'fueltype', 'notrepaired']

# Itera sobre cada columna
for columna in columnas_faltantes:
    # Calcula la moda de la columna dentro de cada grupo definido por las otras columnas
    moda_condicionada = df.groupby(['price', 'power', 'gearbox', 'fueltype'])[columna].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown')
    
    # Rellena los valores faltantes en la columna utilizando la moda condicionada
    for idx, row in df[df[columna].isnull()].iterrows():
        condiciones = (row['price'], row['gearbox'], row['fueltype'], row['power'])
        df.loc[idx, columna] = moda_condicionada.get(condiciones, 'unknown')

**Limpieza de los valores incongruentes en la columna `price`.**

In [7]:
# Identificar filas con valores atípicos en 'price'
valores_atipicos_price = df[df['price'] < 800]

# Filtrar filas con valores válidos en 'price'
valores_validos_price = df[df['price'] >= 800]

# Iterar sobre las filas con valores atípicos en 'price'
for idx, fila_atipica_price in valores_atipicos_price.iterrows():
    # Filtrar filas con características similares a la fila atípica en otras columnas
    filas_similares_price = valores_validos_price[
        (valores_validos_price['brand'] == fila_atipica_price['brand']) &
        (valores_validos_price['model'] == fila_atipica_price['model']) 
        # Agrega más condiciones si es necesario para reflejar las características relevantes
    ]
    # Calcular la mediana de 'price' para filas similares
    mediana_similares_price = filas_similares_price['price'].median()
    # Imputar la mediana como el nuevo valor para la fila atípica en 'price'
    df.loc[idx, 'price'] = mediana_similares_price

# Verificar que el cambio se haya aplicado correctamente
print(df['price'].min())

800.0


**Utilizaremos describe para poder ver que nuestros datos sean congruentes**

In [9]:
# Obtener las estadisticas descriptivas
df_stats = df.describe()
df = df.dropna()
# Mostras las estadisticas
print(df_stats)

               price  registrationyear          power        mileage
count  354368.000000     354294.000000  354369.000000  354369.000000
mean     4842.848546       2003.084077     110.094337  128211.172535
std      4257.820184          7.536162     189.850405   37905.341530
min       800.000000       1910.000000       0.000000    5000.000000
25%      1800.000000       1999.000000      69.000000  125000.000000
50%      3200.000000       2003.000000     105.000000  150000.000000
75%      6490.000000       2008.000000     143.000000  150000.000000
max     20000.000000       2019.000000   20000.000000  150000.000000


# 2.2 Preparación de los datos para los modelos

**Definimos nuestra variable X para las caracteristicas y nuestra variable y como objetivo que es la columna `price`.**

In [10]:
# Dividir los datos en características y objetivo
X = df.drop('price', axis=1)
y = df['price']

In [11]:
# Preprocesamiento para ML
columnas_numericas = X.select_dtypes(include=['int', 'float']).columns
columnas_categoricas = X.select_dtypes(include=['object']).columns

**Dividimos nuestro conjunto de datos en 60 para el entrenamiento, 20 para la prueba y 20 para la validación.**

In [12]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=12345)

In [13]:
X_test, X_valid, y_test, y_valid = train_test_split(X, y, test_size=0.5, random_state=12345)

## Entrenamiento del modelo 

**Entrenaremos un modelo de Regresion Lineal**

In [30]:
# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), columnas_categoricas)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

Root Mean Squared Error: 2820.6994825761685


**Entrenaremos un modelo de Bosque de Regresion**

In [22]:
# Preprocesar los datos antes de dividirlos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor())])  

# Obtener los datos
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target variable

# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir la cuadrícula de parámetros a buscar
param_grid = {
    'regressor__n_estimators': [20, 40, 80],  # Número de árboles en el bosque
    'regressor__max_depth': [2, 4, 8],  # Profundidad máxima de los árboles
}

# Realizar búsqueda de cuadrícula
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_root_mean_squared_error')
inicio_busqueda = time.time()
grid_search.fit(X_train, y_train)
fin_busqueda = time.time()
tiempo_busqueda = fin_busqueda - inicio_busqueda

# Obtener el mejor modelo
best_pipeline = grid_search.best_estimator_

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = best_pipeline.predict(X_test)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Calcular el RMSE
mse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

# Imprimir los mejores parámetros y el RMSE
print("Mejores parámetros encontrados:", grid_search.best_params_)
print("Root Mean Squared Error del mejor modelo:", -grid_search.best_score_) 

print("Tiempo de búsqueda de cuadrícula:", tiempo_busqueda, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")

Root Mean Squared Error: 2091.565397840245
Mejores parámetros encontrados: {'regressor__max_depth': 8, 'regressor__n_estimators': 80}
Root Mean Squared Error del mejor modelo: 2088.585375819942
Tiempo de búsqueda de cuadrícula: 1776.6908972263336 segundos
Velocidad de predicción: 9.114696620207444e-06 segundos por instancia


**Entrenaremos un Modelo LGBMRegressor.**

In [24]:
# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', lgb.LGBMRegressor(**params))])

# Medir tiempo de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_valid)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion


# Entrenar el pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones con el conjunto de prueba
y_pred = pipeline.predict(X_test)

# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

# Imprimir velocidad de predicción y tiempo de entrenamiento
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.089047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [W

**Entrenaremos un modelo de CatBoostRegressor**

In [16]:
# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        # No necesitamos tratar las variables categóricas previamente para CatBoost
    ])

# Crear el pipeline de preprocesamiento y modelo
params = {
    'objective': 'RMSE',  # Usamos 'RMSE' como métrica
    'num_boost_round': 100,  # Número máximo de iteraciones
    'learning_rate': 0.1,
}

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', CatBoostRegressor(**params))])

# Medir tiempo de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_test)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Calcular el RMSE
mse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

# Imprimir velocidad de predicción y tiempo de entrenamiento
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")


0:	learn: 3992.9183177	total: 99.6ms	remaining: 9.86s
1:	learn: 3775.5681707	total: 133ms	remaining: 6.5s
2:	learn: 3581.9663799	total: 165ms	remaining: 5.33s
3:	learn: 3402.5447368	total: 197ms	remaining: 4.72s
4:	learn: 3247.8679759	total: 228ms	remaining: 4.33s
5:	learn: 3113.2088405	total: 260ms	remaining: 4.07s
6:	learn: 3000.3905913	total: 293ms	remaining: 3.89s
7:	learn: 2895.8306931	total: 335ms	remaining: 3.86s
8:	learn: 2809.8773865	total: 369ms	remaining: 3.73s
9:	learn: 2739.6471163	total: 403ms	remaining: 3.62s
10:	learn: 2670.0548735	total: 434ms	remaining: 3.51s
11:	learn: 2612.0556889	total: 465ms	remaining: 3.41s
12:	learn: 2563.8957663	total: 496ms	remaining: 3.32s
13:	learn: 2522.1139796	total: 527ms	remaining: 3.24s
14:	learn: 2486.8600543	total: 559ms	remaining: 3.17s
15:	learn: 2452.8765142	total: 591ms	remaining: 3.1s
16:	learn: 2426.0636718	total: 622ms	remaining: 3.04s
17:	learn: 2401.1095329	total: 652ms	remaining: 2.97s
18:	learn: 2379.7007551	total: 684ms	re

**Entrenaremos un modelo XGBRegressor**

In [17]:
# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', XGBRegressor(objective='reg:squarederror'))])  # 'reg:squarederror' para regresión

# Medir tiempo de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_test)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Calcular el RMSE
mse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

# Imprimir velocidad de predicción y tiempo de entrenamiento
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")

Root Mean Squared Error: 1647.2193735819444
Tiempo de entrenamiento: 27.830370903015137 segundos
Velocidad de predicción: 9.318796879038782e-06 segundos por instancia


## Análisis del modelo

In [33]:
# Definir los resultados de prueba y entrenamiento de cada modelo
resultados = {
    'Modelo': ['XGBoost','LightGBM','Random Forest','CatBoost'],
    'RMSE Testeo': [1647.22, 1765.77, 2091.57, 2177.88],  # RMSE obtenido en prueba para cada modelo
    'Tiempo de Predicción (s)': [9.32, 3.11, 9.11,2.84],  # Tiempo de predicción promedio por instancia en segundos
    'Tiempo de Entrenamiento (s)': [27.83, 10.14,1776.70,3.53]  # Tiempo de entrenamiento en segundos para cada modelo
}

# Crear DataFrame con los resultados
df_resultados = pd.DataFrame(resultados)

# Mostrar la tabla
display(df_resultados)

,Modelo,RMSE Testeo,Tiempo de Predicción (s),Tiempo de Entrenamiento (s)
0,XGBoost,1647.22,9.32,27.83
1,LightGBM,1765.77,3.11,10.14
2,Random Forest,2091.57,9.11,1776.70
3,CatBoost,2177.88,2.84,3.53


In [18]:
# Numero Uno Mejor Modelo Ahora con el conjunto de Validación

# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', XGBRegressor(objective='reg:squarederror'))])  # 'reg:squarederror' para regresión

# Medir tiempo de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_valid)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Calcular el RMSE
mse_valid = mean_squared_error(y_valid, y_pred, squared=False)
print("Root Mean Squared Error:", mse_valid)

# Imprimir velocidad de predicción y tiempo de entrenamiento
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")


Root Mean Squared Error: 1703.730840477373
Tiempo de entrenamiento: 27.944493532180786 segundos
Velocidad de predicción: 9.02035171944469e-06 segundos por instancia


**OBSERVACIONES:** Escogimos este modelo como el mejor de los anteriores para poner aprueba con nuestro conjunto de validación ya que a pesar de que es un poco tardado en su tiempo de entrenamiento y de predicción, posee el menor valor rmse aunque en este momento podemos ver que con el conjunto de validación su rmse aumento casi 200 aunqus su tiempo para entrenar y predecir disminuyo un poco.

In [19]:
# Segundo Mejor modelo con el Conjunto de Validación

# Crear transformadores para preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', lgb.LGBMRegressor(**params))])

# Medir tiempo de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Medir tiempo de predicción
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_valid)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Entrenar el pipeline con los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Realizar predicciones con el conjunto de prueba
y_pred = pipeline.predict(X_valid)

# Evaluar el modelo
mse_valid = mean_squared_error(y_valid, y_pred, squared=False)
print("Root Mean Squared Error:", mse)

# Imprimir velocidad de predicción y tiempo de entrenamiento
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")


[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 wi

**OBSERVACIONES:** Este fue uno de los modelos que menos tardo en entrenarse y para predecir y fue el segundo con un mejor valor rmse, algo interesante que podemos ver es que en este momento este modelo `LGBMRegressor` obtuvo un mejor valor rmse que el modelo XGBRegressor y ademas tiene un mejor tiempo, por lo tanto hasta este momento este es nuestro mejor modelo.

In [26]:
# Tercer Mejor modelo ahora con el Conjunto de Validación

# Preprocesar los datos antes de dividirlos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), columnas_numericas),
        ('cat', OneHotEncoder(), columnas_categoricas)
    ])

# Crear el pipeline de preprocesamiento y modelo con los mejores parámetros
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(max_depth=8, n_estimators=80))])  


# Entrenar el modelo con los datos de entrenamiento
inicio_entrenamiento = time.time()
pipeline.fit(X_train, y_train)
fin_entrenamiento = time.time()
tiempo_entrenamiento = fin_entrenamiento - inicio_entrenamiento

# Predecir con el modelo entrenado
inicio_prediccion = time.time()
y_pred = pipeline.predict(X_valid)
fin_prediccion = time.time()
tiempo_prediccion = fin_prediccion - inicio_prediccion

# Calcular el RMSE
mse_valid = mean_squared_error(y_valid, y_pred, squared=False)
print("Root Mean Squared Error:", mse_valid)

# Imprimir el tiempo de entrenamiento y predicción
print("Tiempo de entrenamiento:", tiempo_entrenamiento, "segundos")
print("Velocidad de predicción:", tiempo_prediccion / len(y_test), "segundos por instancia")

Root Mean Squared Error: 2067.538706726016
Tiempo de entrenamiento: 179.73693752288818 segundos
Velocidad de predicción: 2.3031351159490013e-05 segundos por instancia


**OBSERVACIONES:** Escogimos este modelo para probar con nuestro conjunto de validación debido a que era el tercer modelo con mejor valor rmse aunque el unico detalle con este modelo es que es el mas lento para entrenarse al haber durado mucho mas que los demas.

# Conclusión

Para concluir este proyecto me gustaria destacar tres puntos principales que son los siguientes:
1. Nuesto mejor modelo es el LGBM Regressor con un RMSE de 1647 en el conjunto de validación y un tiempo de entrenamiento de aproximadamente 7.5 segundos y 1 segundo de velocidad de predicción.
2. Siempre es bueno revisar los modelos con el conjunto de validación y testeo para decidir el mejor y no basarnos en solo una prueba con un solo conjunto de datos.
3. Es muy imporante limpiar nuestro conjunto de datos de una forma adecuada y no tener valores nan para poder trabajar con nuestros modelos.